# Vehicle loan default predection

Importing necessary libs and custom functions

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from IPython.display import Image
from sklearn.ensemble import ExtraTreesClassifier

#importing custom functions from functions.py file
from functions import dateconvert
from functions import yrscalc
from functions import time_diff_months
from functions import resample

In [34]:
df = pd.read_csv(r'../Data/train.csv')

In [35]:
df = df.head(10000)


In [ ]:
df.info()

In [36]:
#We can see some columns has wrong data types like date and have missing values in Employment type
df['Date.of.Birth'] = df['Date.of.Birth'].apply(dateconvert,delim='-')
df['DisbursalDate'] = df['DisbursalDate'].apply(dateconvert,delim='-')
df['Employment.Type'].fillna('Unknown',inplace=True)
df.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,1984-01-01 00:00:00+00:00,Salaried,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,1985-07-31 00:00:00+00:00,Self employed,...,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,1985-08-24 00:00:00+00:00,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,1993-12-30 00:00:00+00:00,Self employed,...,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,1977-12-09 00:00:00+00:00,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


In [37]:
#for purpose of labeling in human format
df['labels'] = df['loan_default'].apply(lambda x:'Paid' if x==0 else "default")

In [38]:
#Feature engineering
#We have features some of which might not be useful or have negligible effect on models.
#we also replace string values with integers.
#Date of birth and loan disbursal dates are used to calculate age of person and time since disbursement of loan
emp_type = {'Unknown':0,'Self employed':1,'Salaried':2}

cns_desc = {'No Bureau History Available':0,
            'Not Scored: Sufficient History Not Available':0,
            'Not Scored: Not Enough Info available on the customer':0,
           'Not Scored: No Activity seen on the customer (Inactive)':0,
           'Not Scored: No Updates available in last 36 months':0,
           'Not Scored: Only a Guarantor':1,
           'Not Scored: More than 50 active Accounts found':1,
           'M-Very High Risk':2,
           'L-Very High Risk':3,
           'K-High Risk':4,
            'J-High Risk':5,
            'H-Medium Risk':6,
            'I-Medium Risk':7,
            'F-Low Risk':8,
            'E-Low Risk':9,
            'G-Low Risk':10,
            'C-Very Low Risk':11,
            'A-Very Low Risk':12,
            'D-Very Low Risk':13,
            'B-Very Low Risk':14
           }
df['Age'] = df['Date.of.Birth'].apply(time_diff_months)
df['AVERAGE.ACCT.AGE'] = df['AVERAGE.ACCT.AGE'].apply(yrscalc)
df['CREDIT.HISTORY.LENGTH'] = df['CREDIT.HISTORY.LENGTH'].apply(yrscalc)
df['Age.of.Loan'] = df['DisbursalDate'].apply(time_diff_months)
#combaining to sum of ids
df['No.of.Proofs'] = df[['Aadhar_flag','VoterID_flag','PAN_flag','Driving_flag','Passport_flag','MobileNo_Avl_Flag']].sum(axis=1)

In [39]:
df['PERFORM_CNS.SCORE.DESCRIPTION'] = df['PERFORM_CNS.SCORE.DESCRIPTION'].map(cns_desc)
df['Employment.Type'] = df['Employment.Type'].map(emp_type)


In [ ]:
#Now we can see columns in desired datatypes and 
df.info()

In [47]:
df.describe()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Employment.Type,State_ID,...,Age,Age.of.Loan,No.of.Proofs,Total.ACCTS,Total.Active.ACCTS,Total.Overdue.ACCTS,Total.CurrentBalance,Total.SanctionedAmount,Total.DisbursedAmount,Total.InstalAmount
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
mean,538785.065100,53404.665900,76139.352500,73.264215,49.082900,18806.575600,66.483600,4158.611700,1.349000,6.691100,...,35.435996,1.500890,2.065000,2.122600,0.899600,0.129800,1.426077e+05,1.782786e+05,1.772445e+05,6.699809e+03
std,68430.043267,11881.524803,17203.625425,12.010732,44.029082,3181.370226,21.397531,2147.595063,0.571167,4.560368,...,9.822157,0.093301,0.253735,4.453802,1.726041,0.496766,1.002615e+06,1.148620e+06,1.148055e+06,4.824099e+04
min,417430.000000,13940.000000,39605.000000,17.050000,1.000000,12311.000000,45.000000,4.000000,0.000000,1.000000,...,19.100000,1.400000,2.000000,0.000000,0.000000,0.000000,-1.133510e+05,0.000000e+00,0.000000e+00,0.000000e+00
25%,478743.250000,46702.500000,65984.250000,66.610000,16.000000,16166.000000,45.000000,1872.000000,1.000000,3.000000,...,27.400000,1.400000,2.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,539534.500000,53303.000000,71170.000000,75.000000,36.000000,18080.000000,51.000000,5084.000000,1.000000,5.000000,...,33.200000,1.500000,2.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,600466.500000,59258.250000,79948.500000,82.512500,67.000000,22483.000000,86.000000,6013.000000,2.000000,10.000000,...,42.132500,1.600000,2.000000,2.000000,1.000000,0.000000,2.287025e+04,4.960000e+04,4.841700e+04,1.590000e+03
max,658637.000000,157511.000000,225100.000000,95.000000,152.000000,24787.000000,145.000000,7149.000000,2.000000,18.000000,...,65.600000,1.700000,4.000000,90.000000,31.000000,14.000000,6.229941e+07,8.124306e+07,8.126467e+07,1.948754e+06


In [ ]:
#imbalanced data
fig = px.pie(df, names='labels')
fig.show()

In [ ]:
#Static plot for no dynamic render.
fig.write_image(r"../Images/imbalanced_data.png")
img_bytes = fig.to_image(format="png")
Image(img_bytes)

In [19]:
#feature importance. We drop rudumentry features.
X = df.drop(columns=['UniqueID','loan_default','labels','Date.of.Birth','DisbursalDate','Aadhar_flag','PAN_flag','VoterID_flag','Driving_flag','Passport_flag','MobileNo_Avl_Flag'])
y = df[['loan_default']]

In [20]:
#dealing with imbalanced data
os_X,os_y = resample(X,y,'minority')

In [ ]:
X

In [21]:
#dealing with imbalanced data
#We use ExtraTressClassifer for feture selection which failry robust and also computationaly inexpensive.
os_X,os_y = resample(X,y,'minority')
etc_model = ExtraTreesClassifier(oob_score=True,random_state=27,bootstrap=True,n_estimators=15)
etc_model.fit(os_X,os_y)

C:\Users\kavin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=None,
                     oob_score=True, random_state=27, verbose=0,
                     warm_start=False)

In [22]:
#Craeting a dataframe for a tubular column and sorting them in decending order.
#We can see that features with secondary accounts have negligible. So we combine both primary and secondary account features.
columns = list(X.columns)
pd_feature = pd.DataFrame(data=[columns,etc_model.feature_importances_])
pd_feature = pd_feature.T
pd_feature.columns=['features','scores']
pd_feature.sort_values(['scores'],ascending=False)

,features,scores
6,Current_pincode_ID,0.0905751
32,Age.of.Loan,0.0861097
11,PERFORM_CNS.SCORE.DESCRIPTION,0.075564
10,PERFORM_CNS.SCORE,0.0668777
1,asset_cost,0.0649333
7,Employment.Type,0.0561976
13,PRI.ACTIVE.ACCTS,0.0540078
2,ltv,0.0532654
14,PRI.OVERDUE.ACCTS,0.0457071
0,disbursed_amount,0.0451555


In [ ]:
plt.figure(figsize=(25,25))
fig = px.bar(pd_feature,x='features',y='scores',text='scores', width=900, height=700)
fig.show()

In [ ]:
#non dynamic render.For purpose of being displayed in Github
#Image(filename=r'../Images/feature_score.png')
fig.write_image(r"../Images/feature_score.png")
img_bytes = fig.to_image(format="png")
Image(img_bytes)

In [ ]:
etc_model.oob_score_

In [40]:
#we could see that the secondary account features dont contribute much. We will combanie them with the primary account features.
df['Total.ACCTS'] = df[['PRI.NO.OF.ACCTS','SEC.NO.OF.ACCTS']].sum(axis=1)
df['Total.Active.ACCTS'] = df[['PRI.ACTIVE.ACCTS','SEC.ACTIVE.ACCTS']].sum(axis=1)
df['Total.Overdue.ACCTS'] = df[['PRI.OVERDUE.ACCTS','SEC.OVERDUE.ACCTS']].sum(axis=1)
df['Total.CurrentBalance'] = df[['PRI.CURRENT.BALANCE','SEC.CURRENT.BALANCE']].sum(axis=1)
df['Total.SanctionedAmount'] = df[['PRI.SANCTIONED.AMOUNT','SEC.SANCTIONED.AMOUNT']].sum(axis=1)
df['Total.DisbursedAmount'] = df[['PRI.DISBURSED.AMOUNT','SEC.DISBURSED.AMOUNT']].sum(axis=1)
df['Total.InstalAmount'] = df[['PRIMARY.INSTAL.AMT','SEC.INSTAL.AMT']].sum(axis=1)

In [41]:
X = df.drop(columns=['UniqueID','loan_default','labels','Date.of.Birth','DisbursalDate','Aadhar_flag','PAN_flag','VoterID_flag','Driving_flag','Passport_flag','MobileNo_Avl_Flag','PRI.NO.OF.ACCTS',
                    'SEC.NO.OF.ACCTS','PRI.ACTIVE.ACCTS','SEC.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS','SEC.OVERDUE.ACCTS','PRI.CURRENT.BALANCE','SEC.CURRENT.BALANCE',
                    'PRI.SANCTIONED.AMOUNT','SEC.SANCTIONED.AMOUNT','PRI.DISBURSED.AMOUNT','SEC.DISBURSED.AMOUNT','PRIMARY.INSTAL.AMT','SEC.INSTAL.AMT'])
y = df[['loan_default']]

In [42]:
os_X,os_y = resample(X,y,'minority')
etc_model = ExtraTreesClassifier(oob_score=True,random_state=27,bootstrap=True,n_estimators=15)
etc_model.fit(os_X,os_y)
columns = list(X.columns)
pd_feature = pd.DataFrame(data=[columns,etc_model.feature_importances_])
pd_feature = pd_feature.T
pd_feature.columns=['features','scores']
pd_feature.sort_values(['scores'],ascending=False)

C:\Users\kavin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

C:\Users\kavin\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:523: UserWarning:

Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.

C:\Users\kavin\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:528: RuntimeWarning:

invalid value encountered in true_divide



,features,scores
18,Age.of.Loan,0.0764136
2,ltv,0.0756061
6,Current_pincode_ID,0.0681447
0,disbursed_amount,0.0660995
1,asset_cost,0.065164
17,Age,0.0632853
9,Employee_code_ID,0.0611812
3,branch_id,0.0579569
5,manufacturer_id,0.0572606
4,supplier_id,0.0548823


In [27]:
os_X = os_X.drop(columns=['Employee_code_ID','State_ID','No.of.Proofs','branch_id'])

In [46]:
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier
brc_model = BalancedRandomForestClassifier(n_estimators=10)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=27)
scores = cross_val_score(brc_model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print(f'Mean AUC_ROC score : {scores.mean()}')

Mean AUC_ROC score : 0.6068006311787534


In [43]:
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier
brc_model = BalancedRandomForestClassifier(n_estimators=10)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=27)
scores = cross_val_score(brc_model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print(f'Accuracy : {scores.mean()}')

Accuracy : 0.5786999999999999


In [67]:
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=15)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=27)
scores = cross_val_score(rf_model, os_X, os_y, scoring='roc_auc', cv=cv, n_jobs=-1)
print(f'Mean AUC_ROC score : {scores.mean()}')

Mean AUC_ROC score : 0.9160831035753463


In [66]:
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=15)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=27)
scores = cross_val_score(rf_model, os_X, os_y, scoring='accuracy', cv=cv, n_jobs=-1)
print(f'Accuracy : {scores.mean()}')

Accuracy : 0.8544854360907487


In [ ]:
from sklearn.metrics import classification_report
c_report = classification_report(test_targets,y_pred_logmodel,target_names=['Negative','Positive'])

In [48]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=rf_model,verbose=5)
rfe_selector.fit(os_X, os_y)
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

C:\Users\kavin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
13 selected features


In [49]:
rfe_feature

['disbursed_amount',
 'asset_cost',
 'ltv',
 'branch_id',
 'supplier_id',
 'manufacturer_id',
 'Current_pincode_ID',
 'Employment.Type',
 'State_ID',
 'Employee_code_ID',
 'PERFORM_CNS.SCORE',
 'Age',
 'Age.of.Loan']

In [62]:
os_X1 = df[['disbursed_amount',
 'asset_cost',
 'ltv',
 'branch_id',
 'supplier_id',
 'manufacturer_id',
 'Current_pincode_ID',
 'Employment.Type',
 'State_ID',
 'Employee_code_ID',
 'PERFORM_CNS.SCORE',
 'Age',
 'Age.of.Loan']]
os_y1 = df[['loan_default']]
os_X1,os_y1 = resample(os_X1,os_y1,'minority')

In [63]:
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=10)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=27)
scores = cross_val_score(rf_model, os_X1, os_y1, scoring='roc_auc', cv=cv, n_jobs=-1)
print(f'Mean AUC_ROC score : {scores.mean()}')

Mean AUC_ROC score : 0.906017382329971


In [65]:
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=10)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=27)
scores = cross_val_score(rf_model, os_X1, os_y1, scoring='accuracy', cv=cv, n_jobs=-1)
print(f'Accuracy : {scores.mean()}')

Accuracy : 0.844002735392018
